Deep Learning
=============
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (18724, 28, 28, 1) (18724, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [13]:
batch_size = 16
patch_size = 5
depth1 = 32
depth2 = 64
num_hidden = 100

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  keep_prob = tf.placeholder("float")
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth1], stddev=0.1)) # 16 feature extraction in 1st conv.
  layer1_biases = tf.Variable(tf.zeros([depth1])) # each feature has only one shared bias.

  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth1, depth2], stddev=0.1)) # 32 channels, and 64 feature extraction.
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth2]))

  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size/4 * image_size/4 * depth2, num_hidden], stddev=0.1)) # Fully connected layer.
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_hidden], stddev=0.1)) # Fully connected layer.
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  
  layer5_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1)) # Output layer connecting fully connected layer to the output layer.
  layer5_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    # First convolutional layer with maxpooling.
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases) # ReLU is used for functional mapping. It is cheaper and faster.
    # Apply pooling to reduce the size.
    pool_1 = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # Second convolutional layer with maxpooling.
    conv = tf.nn.conv2d(pool_1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    # Max pool.
    pool_2 = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # Reshape the output so that we can connect with the fully connected layer.
    shape = pool_2.get_shape().as_list()
    reshape = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    
    # Full connected layer with ReLU.
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    hidden = tf.nn.relu(tf.matmul(hidden, layer4_weights) + layer4_biases)
    
    hidden_dropout = tf.nn.dropout(hidden, keep_prob)
    
    return tf.matmul(hidden_dropout, layer5_weights) + layer5_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  # Apply softmax for the output values to get the probability distribution of values.
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  global_step = tf.Variable(0, trainable=False)
  starter_learning_rate = 0.05
  learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                           200, 0.96, staircase=True)
  #optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step) 
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss) 
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [14]:
num_steps = 4001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(feed_dict={keep_prob: 1.0}), valid_labels)
    
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(feed_dict={keep_prob: 1.0}), test_labels)

Initialized
Minibatch loss at step 0 : 8.6322
Minibatch accuracy: 0.0%
Validation accuracy: 10.4%
Minibatch loss at step 50 : 2.22179
Minibatch accuracy: 12.5%
Validation accuracy: 13.9%
Minibatch loss at step 100 : 1.99232
Minibatch accuracy: 25.0%
Validation accuracy: 28.6%
Minibatch loss at step 150 : 2.08167
Minibatch accuracy: 25.0%
Validation accuracy: 50.8%
Minibatch loss at step 200 : 1.14709
Minibatch accuracy: 56.2%
Validation accuracy: 62.1%
Minibatch loss at step 250 : 0.904511
Minibatch accuracy: 87.5%
Validation accuracy: 67.0%
Minibatch loss at step 300 : 0.94108
Minibatch accuracy: 75.0%
Validation accuracy: 73.9%
Minibatch loss at step 350 : 1.1309
Minibatch accuracy: 75.0%
Validation accuracy: 65.7%
Minibatch loss at step 400 : 0.763829
Minibatch accuracy: 75.0%
Validation accuracy: 77.2%
Minibatch loss at step 450 : 1.02073
Minibatch accuracy: 81.2%
Validation accuracy: 67.6%
Minibatch loss at step 500 : 0.719261
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
M

---
Results
---------
* 2 (convolution, maxpool, 32 feature) + (fully connected, softmax, 64 neurons) = 90.8%
* 2 (convolution, maxpool, 32 feature) + (fully connected, softmax, 100 neurons) = 91.3%
* 2 (convolution, maxpool, 32 feature) + (fully connected, 100 neurons) + (fully connected, softmax, 100 neurons) = 91.2%
* (convolution, maxpool, 32 feature) + (convolution, maxpool, 64 feature) + (fully connected, softmax, 100) = 90.7%
* (convolution, maxpool, 32 feature) + (convolution, maxpool, 64 feature) + (fully connected, softmax, dropout, 100) + SGD = 88.1%
* (convolution, maxpool, 32 feature) + (convolution, maxpool, 64 feature) + (fully connected, 100) +  (fully connected, softmax, dropout, 100) + AdamOptimizer + 5001 steps = 89.2%
* (convolution, maxpool, 32 feature) + (convolution, maxpool, 64 feature) + (fully connected, 100) +  (fully connected, softmax, 100) + SGD + 5001 steps = 93.3%
---